In [0]:
import datetime

ts = datetime.datetime.now()
integ_name='nyc_moto_silver_logging'

spark.sql(f"insert into nyc_moto_db_logs.nyc_moto_vehicles_log_header (date_created, integration_name, log_status, date_completed) values ('{ts}','{integ_name}','0',null)")

log_df = spark.sql(f"select log_id from nyc_moto_db_logs.nyc_moto_vehicles_log_header where date_created == '{ts}'")
log_id = log_df.collect()[0][0]

In [0]:
#check for uniqueness of Collision ID for the current batch
df = spark.sql("select count(distinct collision_id) != count(collision_id) from nyc_moto_db_bronze.nyc_moto_vehicles_bronze where batch_id = ''")

if df.collect()[0][0]: 
    spark.sql(f"UPDATE nyc_moto_db_logs.nyc_moto_vehicles_log_header SET log_status = '-1', date_completed = current_timestamp WHERE log_id == '{log_id}'")
    spark.sql(f"insert into nyc_moto_db_logs.nyc_moto_vehicles_log_details (date_created, log_id, log_message, log_value) values (current_timestamp, '{log_id}','Collision IDs in bronze batch are not unique','0')")
    raise Exception("Collision IDs in bronze batch are not unique")
    

In [0]:
import uuid

uuid = uuid.uuid4()

In [0]:
spark.sql(f"UPDATE nyc_moto_db_bronze.nyc_moto_vehicles_bronze SET batch_id = '{uuid}' where batch_id = ''")

Out[14]: DataFrame[num_affected_rows: bigint]

In [0]:
spark.sql(f"""CREATE OR REPLACE TEMPORARY VIEW batch AS SELECT 
coalesce(trim(both from crash_date),'01/01/1900') as crash_date, 
lpad(trim(both from coalesce(crash_time, '00:00')),5,'0') AS crash_time,
coalesce(trim(both from borough),'') as borough,
coalesce(trim(both from zip_code),'') as zip_code,
coalesce(trim(both from latitude),'') as latitude,
coalesce(trim(both from longitude),'') as longitude,
coalesce(trim(both from loc),'') as loc,
coalesce(trim(both from on_street_name),'') as on_street_name,
coalesce(trim(both from cross_street_name),'') as cross_street_name,
coalesce(trim(both from off_street_name),'') as off_street_name,
coalesce(persons_injured,0) as persons_injured,
coalesce(persons_killed,0) as persons_killed,
coalesce(pedestrians_injured,0) as pedestrians_injured,
coalesce(pedestrians_killed,0) as pedestrians_killed,
coalesce(cyclists_injured,0) as cyclists_injured,
coalesce(cyclists_killed,0) as cyclists_killed,
coalesce(motorists_injured,0) as motorists_injured,
coalesce(motorists_killed,0) as motorists_killed,
coalesce(trim(both from contributing_factor_vehicle_1),'') as contributing_factor_vehicle_1,
coalesce(trim(both from contributing_factor_vehicle_2),'') as contributing_factor_vehicle_2,
coalesce(trim(both from contributing_factor_vehicle_3),'') as contributing_factor_vehicle_3,
coalesce(trim(both from contributing_factor_vehicle_4),'') as contributing_factor_vehicle_4,
coalesce(trim(both from contributing_factor_vehicle_5),'') as contributing_factor_vehicle_5,
trim(both from collision_id) as collision_id,
coalesce(trim(both from vehicle_type_code_1),'') as vehicle_type_code_1,
coalesce(trim(both from vehicle_type_code_2),'') as vehicle_type_code_2,
coalesce(trim(both from vehicle_type_code_3),'') as vehicle_type_code_3,
coalesce(trim(both from vehicle_type_code_4),'') as vehicle_type_code_4,
coalesce(trim(both from vehicle_type_code_5),'') as vehicle_type_code_5,
batch_id,
current_timestamp as process_date
FROM nyc_moto_db_bronze.nyc_moto_vehicles_bronze 
WHERE batch_id = '{uuid}' AND zip_code is not null AND collision_id is not null""")

Out[15]: DataFrame[]

In [0]:
spark.sql("""MERGE INTO nyc_moto_db_silver.nyc_moto_vehicles_silver AS s USING batch AS b 
on s.collision_id = b.collision_id 
WHEN MATCHED THEN UPDATE SET * 
WHEN NOT MATCHED THEN INSERT *""")

Out[16]: DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"UPDATE nyc_moto_db_bronze.nyc_moto_vehicles_bronze SET process_date = current_timestamp where batch_id = '{uuid}'")

Out[17]: DataFrame[num_affected_rows: bigint]

In [0]:
spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW zip_errors AS SELECT * FROM nyc_moto_db_bronze.nyc_moto_vehicles_bronze WHERE batch_id = '{uuid}' AND zip_code is null") 
spark.sql("INSERT INTO nyc_moto_db_logs.nyc_moto_vehicles_errors_bronze_zip SELECT * FROM zip_errors")
num_zip = spark.sql("select count(*) from zip_errors")
num_zip_errors = num_zip.collect()[0][0]
spark.sql(f"insert into nyc_moto_db_logs.nyc_moto_vehicles_log_details (date_created, log_id, log_message, log_value) values (current_timestamp, '{log_id}','number of records written to errors_bronze_zip','{num_zip_errors}')")

Out[18]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
num_batch = spark.sql("select count(*) from batch")
num_batch_records = num_batch.collect()[0][0]
spark.sql(f"insert into nyc_moto_db_logs.nyc_moto_vehicles_log_details (date_created, log_id, log_message, log_value) values (current_timestamp, '{log_id}','number of records written to nyc_moto_silver','{num_batch_records}')")

Out[19]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"UPDATE nyc_moto_db_logs.nyc_moto_vehicles_log_header SET log_status = '1', date_completed = current_timestamp WHERE log_id == '{log_id}'")

Out[20]: DataFrame[num_affected_rows: bigint]